In [3]:
import pandas as pd

df_train = pd.read_csv("../data/train/train.csv")
df_test = pd.read_csv("../data/processed/df_test_proce.csv")

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

X = df_train.drop('price_range', axis=1)
y = df_train['price_range']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

#test_ids = df_test['id']
#df_test = df_test.drop('id', axis=1)

test_data_scaled = scaler.transform(df_test)

In [5]:
import os
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression  # Importación correcta
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# Ajustar la advertencia de núcleos de CPU
#os.environ["LOKY_MAX_CPU_COUNT"] = "4"  # Ajusta según el número de núcleos de tu máquina

models = {
    'RandomForestClassifier': RandomForestClassifier(random_state=42),
    'GradientBoostingClassifier': GradientBoostingClassifier(random_state=42),
    'AdaBoostClassifier': AdaBoostClassifier(algorithm='SAMME', random_state=42),  # SAMME para evitar FutureWarning
    'ExtraTreesClassifier': ExtraTreesClassifier(random_state=42),
    'BaggingClassifier': BaggingClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(random_state=42),  # Importación corregida
    'SVC': SVC(random_state=42),
    'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42),
    'XGBClassifier': XGBClassifier(random_state=42),
    'CatBoostClassifier': CatBoostClassifier(random_state=42, verbose=False),
    'LGBMClassifier': LGBMClassifier(random_state=42, verbose=-1),
}

with open('classification_reports.txt', 'w') as f:
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        f.write(f"{name}\n")
        f.write(classification_report(y_val, y_pred))
        f.write("--------------------------------\n")
